In [1]:
import gdown
import pickle as pkl
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', category=FutureWarning)

In [2]:
# Google Collab
# from google.colab import drive
# drive.mount('/content/drive')
# !gdown https://drive.google.com/file/d/1r-kWDZe3IoFTQm1gBOZCm0w_uoAhreZE/view?usp=sharing --fuzzy
# !gdown https://drive.google.com/file/d/1qiikMTylH8d8ic37ehM_CDPDmZIbQ7Iy/view?usp=sharing --fuzzy
# !gdown https://drive.google.com/file/d/1lyR8C1SCs6jD4aIPvzPam2iyk8bmGCdC/view?usp=sharing --fuzzy
# !gdown https://drive.google.com/file/d/1PG1B-XrZQ4SAhmSotpUilPKCUgZTQmFZ/view?usp=sharing --fuzzy
# !gdown https://drive.google.com/file/d/1KNXsdZkXeIf5qglUV5AS_2_o9yeTevgU/view?usp=sharing --fuzzy
# directory = ''

# Local: Data already in folder Clean Dataset
directory = "CleanDataset/"

In [3]:
X_train = pkl.load(open(f'{directory}X_train.pkl', 'rb'))
y_train = pkl.load(open(f'{directory}y_train.pkl', 'rb'))
X_train

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
2340,-0.115291,-0.057902,-0.562189,0.200000,-0.011261,-0.500000,-0.422719,0.165815,0.0,-0.456189,-0.514625,0.0,-0.237245
11980,0.117094,0.736939,-0.597015,-0.877193,-0.022342,-0.333333,-0.093110,-0.482334,0.0,-0.357904,0.194790,0.0,-0.959184
16317,-1.704629,-0.305567,0.089552,0.112281,0.055946,0.166667,1.461825,0.183726,0.0,0.148354,0.519506,0.0,-0.655612
14155,-0.664818,-0.344940,-0.522388,1.045614,2.824505,-0.166667,1.871508,0.722172,0.0,0.569309,0.336038,0.0,-0.489796
12999,0.427800,0.481642,0.238806,-0.221053,0.230811,0.333333,-0.284916,0.747359,0.0,0.159481,-1.370530,-1.0,-0.130102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8135,1.206107,0.110541,0.412935,0.540351,-0.001081,-0.166667,-0.065177,-0.786259,0.0,-0.294854,0.298105,0.0,0.505102
16001,0.925120,-0.357521,-1.189055,0.589474,-0.022342,-0.666667,0.102421,0.185405,0.0,-0.272601,1.404538,0.0,-0.165816
9199,1.649397,-0.283985,0.567164,0.340351,19.437117,0.666667,1.899441,-0.543903,-1.0,2.143718,-0.666615,0.0,0.747449
5649,-0.395735,0.489538,0.373134,-0.543860,7.878559,-1.000000,0.018622,-0.383265,0.0,0.090867,1.473716,0.0,0.696429


In [4]:
y_train

2340     25
11980    61
16317    29
14155    51
12999    68
         ..
8135     48
16001    64
9199     26
5649     51
4763     62
Name: song_popularity, Length: 7144, dtype: int64

## Modelling

### 1. ANN Modelling

In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=23)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5715, 13), (1429, 13), (5715,), (1429,))

In [6]:
model = Sequential()

# input layer
model.add(Input(shape=(13,)))

# hidden layers
model.add(Dense(32,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))

# output layer
model.add(Dense(1,activation='linear'))

model.compile(optimizer='sgd', loss='mse', 
              metrics=[tf.keras.metrics.RootMeanSquaredError(),
                       tf.keras.metrics.MeanAbsoluteError(),
                       tf.keras.metrics.MeanAbsolutePercentageError(), 
                       tf.keras.metrics.R2Score()])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,169 (340.50 KB)

 Trainable params: 87,169 (340.50 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 80
)

import os
checkpoint_filepath = 'Model/ANN_best_model.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_filepath)
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    save_weights_only = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75,
                              patience=5, min_lr=1e-9)

callbacks = [cb_checkpoint,early_stopping, reduce_lr]

In [8]:
model.fit(x=X_train.values,y=y_train.values,validation_data=(X_val,y_val),batch_size=256,epochs=400, callbacks=callbacks)

Epoch 1/400
19/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan - mean_absolute_error: nan - mean_absolute_percentage_error: nan - r2_score: nan - root_mean_squared_error: nan                         
Epoch 1: val_loss did not improve from inf
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: nan - mean_absolute_error: nan - mean_absolute_percentage_error: nan - r2_score: nan - root_mean_squared_error: nan - val_loss: nan - val_mean_absolute_error: nan - val_mean_absolute_percentage_error: nan - val_r2_score: nan - val_root_mean_squared_error: nan - learning_rate: 0.0100
Epoch 2/400
21/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: nan - mean_absolute_error: nan - mean_absolute_percentage_error: nan - r2_score: nan - root_mean_squared_error: nan
Epoch 2: val_loss did not improve from inf
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: nan - mean_absolute_error: nan - mean_absolute_percentage_error: nan - r2_score: nan - root_mean_squared_error: nan - val_loss: nan - val_mean_absolute_error: n

In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error
model.load_weights(checkpoint_filepath)
y_predicted = model.predict(X_val)

r2 = r2_score(y_val,y_predicted)
mae = mean_absolute_error(y_val,y_predicted)
rmse = root_mean_squared_error(y_val,y_predicted)
mse = mean_squared_error(y_val,y_predicted)

print('r2 score:',r2)
print('mae score:',mae)
print('rmse score:',rmse)
print('mse score:',mse)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
r2 score: 0.028206229209899902
mae score: 15.931185261197154
rmse score: 19.806161418311312
mse score: 392.2840301282036


In [10]:
model.save('Model/ANN_regressor.keras')